# Use Tools to Search PubMed Central for Scientific Articles

In this notebook, you'll create a research agent using Strands that can query the PubMed Central (PMC) journal database for information about scientific discoveries.

## 1. Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude Sonnet 4 model on Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit

## 2. Basic Prompt without Context

To begin, we'll create a basic agent and see how well it can answer a scientific question without any additional context.

In [ ]:
from strands import Agent

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
QUERY = "How safe and effective are GLP-1 drugs for long term use?"

SYSTEM_PROMPT = """
    You are a specialized life science research agent. Your role is to:
    1. Search PubMed Central for medical papers related to the query
    2. Extract and summarize the most relevant clinical findings
    3. Identify key research groups and methodologies
    4. Return structured, well-cited information with PMCID references
    """

# Initialize your agent
agent = Agent(system_prompt=SYSTEM_PROMPT, model=MODEL_ID)

# Send a message to the agent
response = agent(QUERY)

Copy and paste a few of the PMCIDs in the previous cell into the [PMC web search](https://pmc.ncbi.nlm.nih.gov/). Notice anything unusual? They likely point to completely unrelated resources! Without additional context, LLMs will do their best to generate IDs that seem convincing - they may even return real IDs included in their training data. However, if we want our agent to consistently return accurate, up-to-date results we need to provide it with a tool.

## 3. Search PMC for Scientific Abstracts

Let's see if we can improve the performance of our agent by giving it a tool. To start, we've created a custom tool called `search_pmc_tool` that uses the PMC API to identify relevant scientific article abstracts. This tool has some special features to help the agent focus on the most relevant articles:

- It limits the search to only articles licensed for commercial use
- For each article in the search results, the tool calculates how many OTHER articles include it as a reference. These are likely to be the most impactful and valuable to the agent

You can look at the `search_pmc_tool` code at `search_pmc.py`.

In [ ]:
from strands import Agent
from search_pmc import search_pmc_tool

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
QUERY = "How safe and effective are GLP-1 drugs for long term use?"

SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

1. Use search_pmc_tool to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 2 years"[dp] for recent work.
2. Extract and summarize the most relevant clinical findings.
3. Return structured, well-cited information with PMC ID references.
4. Return URL links associated with PMCID references
"""


# Initialize your agent
agent = Agent(system_prompt=SYSTEM_PROMPT, tools=[search_pmc_tool], model=MODEL_ID)

# Send a message to the agent
response = agent(QUERY)

The additional information makes the agent response much more detailed. Try [searching](https://pmc.ncbi.nlm.nih.gov/) for the PMCIDs again. This time they should link to the correct articles.

## 4. Deploy to Amazon Bedrock AgentCore Runtime

In this step, we'll deploy the agent definition found in the `agent` folder to Amazon Bedrock AgentCore runtime. Let's start by taking a look at our agent code. Notice the `@app.entrypoint` decorator on the `strands_agent_bedrock` function. This is how we tell the AgentCore Runtime how to run the agent.

In [ ]:
%pycat agent.py

In [ ]:
import boto3
from bedrock_agentcore_starter_toolkit import Runtime

ssm = boto3.client("ssm")

agentcore_runtime = Runtime()
agentcore_runtime.configure(
    agent_name="pmc_abstract_agent",
    auto_create_ecr=True,
    execution_role=ssm.get_parameter(
        Name="/deep-research-workshop/agentcore-runtime-role-arn"
    )["Parameter"]["Value"],
    entrypoint="agent.py",
    memory_mode="NO_MEMORY",
    requirements_file="requirements.txt",
)

In [ ]:
agentcore_runtime.launch(auto_update_on_conflict=True)

In [ ]:
agentcore_runtime.invoke(
    {"prompt": "How safe and effective are GLP-1 drugs for long term use?"}
)

## 5. (Optional) Interact with agent using AgentCore Chat

Follow these steps to open an interactive chat session with your new agent.

1. Open a command line terminal in your notebook environment.
2. Navigate to the project root folder (where `pyproject.toml` is located).
3. Run `pip install .` to install the workshop tools including the chat CLI.
4. Run `agentcore-chat` to launch the CLI.
5. Select the `pmc_abstract_agent` by typing its name or index in the terminal and press Enter.
6. Ask your question at the `You:` prompt and press Enter.


## 6. (Optional) Clean Up

Run the next notebook cell to delete the AgentCore runtime environment.

In [ ]:
import boto3

agentcore_client = boto3.client("bedrock-agentcore-control")
agent_status = agentcore_runtime.status()

agentcore_client.delete_agent_runtime(agentRuntimeId=agent_status.config.agent_id)